In [1]:
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install openpyxl




[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip3 install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip3 install --upgrade pip


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import time
from datetime import datetime, timedelta
import pandas as pd
import os
from selenium.common.exceptions import StaleElementReferenceException

# 시간 정보를 날짜로 변환하는 함수
def convert_time_to_date(time):
    now = datetime.now()
    
    if "분 전" in time:
        return now.strftime("%Y-%m-%d")
    elif "시간 전" in time:
        hours_ago = int(time.split("시간 전")[0])
        return (now - timedelta(hours=hours_ago)).strftime("%Y-%m-%d")
    elif "일 전" in time:
        days_ago = int(time.split("일 전")[0])
        return (now - timedelta(days=days_ago)).strftime("%Y-%m-%d")
    elif "주 전" in time:
        weeks_ago = int(time.split("주 전")[0])
        return (now - timedelta(weeks_ago)).strftime("%Y-%m-%d")
    else:
        return time

# 크롤링 함수
def crawl_news(query, media):
    options = Options()
    options.add_argument("start-maximized")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    print(f"'{query}' 검색에 대한 브라우저를 열었습니다.")
    
    results = []
    
    try:
        # 네이버 뉴스 페이지로 이동
        url = f"https://search.naver.com/search.naver?where=news&query={query}&sm=tab_opt&sort=1&photo=0&field=0&pd=3&ds=2019.08.27&de=2024.08.27&docid=&related=0&mynews=1&office_type=1&office_section_code=2&news_office_checked=1437&nso=so%3Add%2Cp%3Afrom20190827to20240827&is_sug_officeid=0&office_category=0&service_area=0"
        driver.get(url)
        print(f"네이버 뉴스 페이지로 이동했습니다. 검색어: {query}")

        # 첫 번째 기사가 로드될 때까지 대기
        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".list_news .bx")))
        print("첫 번째 기사 로드 완료")

        # 스크롤이 끝까지 내려갔는지 확인하면서 스크롤을 반복적으로 내림
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        print("스크롤 완료. 모든 기사가 로드되었습니다.")

        articles = driver.find_elements(By.CSS_SELECTOR, ".list_news .bx")

        for i, article in enumerate(articles):
            try:
                print(f"기사 {i+1} 크롤링 중...")
                title = article.find_element(By.CSS_SELECTOR, ".news_tit").text
                naver_link = article.find_element(By.CSS_SELECTOR, "div.info_group a[href^='https://n.news.naver.com']").get_attribute("href")

                time_elements = article.find_elements(By.CSS_SELECTOR, "span.info")
                time_text = time_elements[1].text if len(time_elements) > 1 else time_elements[0].text if len(time_elements) == 1 else "시간 정보 없음"
                time_parsed = convert_time_to_date(time_text)

                driver.execute_script("window.open('');")  # 새 탭 열기
                driver.switch_to.window(driver.window_handles[-1])  # 새 탭으로 전환
                driver.get(naver_link)

                WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, "article#dic_area")))
                content = driver.find_element(By.CSS_SELECTOR, "article#dic_area").text

                # 따옴표 추가 없이 결과 저장
                results.append({
                    "title": title,
                    "naverLink": naver_link,
                    "time": time_parsed,
                    "keyword": query,
                    "media": media,
                    "content": content
                })

                print("기사 본문 크롤링 완료")
                driver.close()  # 현재 탭 닫기
                driver.switch_to.window(driver.window_handles[0])  # 다시 검색 결과 탭으로 전환

            except StaleElementReferenceException as e:
                print(f"StaleElementReferenceException 발생 (index {i}): {e}. 다음 기사로 넘어갑니다.")
                continue  # 오류 발생 시 해당 기사를 건너뛰고 다음 기사로 넘어갑니다.
            except Exception as e:
                print(f"기사 크롤링 중 오류 발생 (index {i}):", e)
                continue

        print(f"'{query}'에 대한 크롤링 완료. 크롤링한 기사 수: {len(results)}")
        return results

    except Exception as e:
        print("Selenium 작업 중 오류 발생:", e)
        return results  # 현재까지 크롤링된 결과를 반환

    finally:
        # 브라우저를 닫지 않도록 주석 처리
        # driver.quit()
        print(f"'{query}' 검색에 대한 브라우저를 닫지 않습니다. (디버깅용)")

# 검색어 설정
query = '"인사" "정책"'
media = "JTBC"  # 미디어 명시적으로 지정

# 결과 저장할 폴더 경로
folder_name = f"{media}_{query.replace(' ', '_').replace('\"', '')}"
os.makedirs(folder_name, exist_ok=True)

# 명시적으로 지정한 파일 이름 설정
file_name = "JTBC-인사,정책.csv"  # 원하는 파일 이름으로 수정

# 검색어에 대해 크롤링 수행 및 CSV로 저장
print(f"===== '{query}' 검색에 대한 크롤링 시작 =====")
news_data = crawl_news(query, media)

if news_data:
    df = pd.DataFrame(news_data)
    
    # 지정된 파일 이름으로 CSV에 저장
    file_path = os.path.join(folder_name, file_name)
    
    # 파일 열고 기사 수 추가
    with open(file_path, mode='w', encoding='utf-8-sig', newline='') as file:
        file.write(f"# 총 {len(news_data)}개의 기사를 크롤링했습니다.\n")  # 주석 형태로 기사 수 추가
        df.to_csv(file, index=False)

    print(f"'{query}' 검색 결과가 '{file_path}'에 저장되었습니다.")
else:
    print(f"'{query}' 검색에서 크롤링된 데이터가 없습니다.")
print(f"===== '{query}' 검색에 대한 크롤링 완료 =====\n")


===== '"인사" "정책"' 검색에 대한 크롤링 시작 =====
'"인사" "정책"' 검색에 대한 브라우저를 열었습니다.
네이버 뉴스 페이지로 이동했습니다. 검색어: "인사" "정책"
첫 번째 기사 로드 완료
스크롤 완료. 모든 기사가 로드되었습니다.
기사 1 크롤링 중...
기사 본문 크롤링 완료
기사 2 크롤링 중...
기사 본문 크롤링 완료
기사 3 크롤링 중...
기사 본문 크롤링 완료
기사 4 크롤링 중...
기사 본문 크롤링 완료
기사 5 크롤링 중...
기사 본문 크롤링 완료
기사 6 크롤링 중...
기사 본문 크롤링 완료
기사 7 크롤링 중...
기사 본문 크롤링 완료
기사 8 크롤링 중...
기사 본문 크롤링 완료
기사 9 크롤링 중...
기사 본문 크롤링 완료
기사 10 크롤링 중...
기사 본문 크롤링 완료
기사 11 크롤링 중...
기사 본문 크롤링 완료
기사 12 크롤링 중...
기사 본문 크롤링 완료
기사 13 크롤링 중...
기사 본문 크롤링 완료
기사 14 크롤링 중...
기사 본문 크롤링 완료
기사 15 크롤링 중...
기사 본문 크롤링 완료
기사 16 크롤링 중...
기사 본문 크롤링 완료
기사 17 크롤링 중...
기사 본문 크롤링 완료
기사 18 크롤링 중...
기사 본문 크롤링 완료
기사 19 크롤링 중...
기사 본문 크롤링 완료
기사 20 크롤링 중...
기사 본문 크롤링 완료
기사 21 크롤링 중...
기사 본문 크롤링 완료
기사 22 크롤링 중...
기사 본문 크롤링 완료
기사 23 크롤링 중...
기사 본문 크롤링 완료
기사 24 크롤링 중...
기사 본문 크롤링 완료
기사 25 크롤링 중...
기사 본문 크롤링 완료
기사 26 크롤링 중...
기사 본문 크롤링 완료
기사 27 크롤링 중...
기사 본문 크롤링 완료
기사 28 크롤링 중...
기사 본문 크롤링 완료
기사 29 크롤링 중...
기사 본문 크롤링 완료
기사 30 크롤링 중...
기사 본문 크롤링 완료
기사 31 크롤링 중...
기사 본문 크롤링 완